In [1]:
# Load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np
from sqlalchemy import create_engine
engine = create_engine("sqlite:///master_db.sqlite",echo=False)

In [2]:
exhist = pd.read_sql_query('select * from exhist',con=engine)
exhist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689982 entries, 0 to 689981
Data columns (total 13 columns):
extract_point            689982 non-null object
extraction_tons          689982 non-null int64
datestring               689982 non-null int64
total_extraction_tons    689982 non-null int64
reserves                 689982 non-null float64
region                   689982 non-null object
avg_sA                   689982 non-null float64
exhaustion               689982 non-null float64
zone                     689982 non-null object
sector                   689982 non-null object
daily_total_extracted    689982 non-null int64
ep_daily_pct             689982 non-null float64
ep_datestring            689982 non-null object
dtypes: float64(4), int64(4), object(5)
memory usage: 68.4+ MB


In [3]:
date_month = []
date_year = []
datestrings = exhist.datestring.astype(str)
for i in range(len(datestrings)):
    date_month.append(datestrings[i][4:6])
    date_year.append(datestrings[i][:4])
exhist['month'] = date_month
exhist['year'] = date_year
exhist['year_month'] = exhist['year']+exhist['month']
exhist.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602


In [4]:
ep_region = []
for i in range(len(exhist)):
    if exhist['region'][i] == '7210A':
        ep_region.append('rU')
    elif exhist['region'][i] == '7210M':
        ep_region.append('rV')
    elif exhist['region'][i] == '7210N':
        ep_region.append('rW')
    elif exhist['region'][i] == '7700NW':
        ep_region.append('rY')
    elif exhist['region'][i] == '7700SW':
        ep_region.append('rZ')
    else:
        ep_region.append('rX')
len(ep_region)

689982

In [5]:
histdf = exhist.copy()
histdf['region'] = ep_region
histdf.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month
0,611703,405,20151231,405,76860.0,rZ,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512
1,611703,424,20160105,829,76860.0,rZ,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601
2,611703,463,20160128,1292,76860.0,rZ,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601
3,611703,81,20160223,1373,76860.0,rZ,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602
4,611703,54,20160224,1427,76860.0,rZ,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602


In [6]:
# adding wtd averages for summations in grouping
histdf['average_exhaustion'] = histdf['exhaustion']*histdf['extraction_tons']/histdf['daily_total_extracted']
histdf['ep_sA_reserves'] = histdf['avg_sA']*histdf['reserves']
histdf['region_zone'] = histdf['region']+histdf['zone']

In [7]:
histdf.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month,average_exhaustion,ep_sA_reserves,region_zone
0,611703,405,20151231,405,76860.0,rZ,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512,0.006224,15591.600011,rZ611
1,611703,424,20160105,829,76860.0,rZ,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601,0.016071,15591.600011,rZ611
2,611703,463,20160128,1292,76860.0,rZ,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601,0.034697,15591.600011,rZ611
3,611703,81,20160223,1373,76860.0,rZ,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602,0.006916,15591.600011,rZ611
4,611703,54,20160224,1427,76860.0,rZ,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602,0.004788,15591.600011,rZ611


In [8]:
month_days_total = pd.read_csv('Resources/monthly_composite.csv',index_col=False)
month_days = month_days_total[['year_month','run_days']]
month_days.to_csv('Resources/month_days.csv',index=False)

In [9]:
def ep_agg(x):
    column_names = {
        'zone_total_extracted': x['total_extraction_tons'].sum(),
        'zone_reserves': x['reserves'].sum(),
        'zone_percent_extracted_total': x['ep_daily_pct'].sum(),
        'ep_sA_reserves': x['ep_sA_reserves'].sum()
    }
    return pd.Series(column_names,index=['zone_total_extracted','zone_reserves','zone_percent_extracted_total','ep_sA_reserves'])
zone_months = histdf.groupby(['region_zone','year_month']).apply(ep_agg).reset_index(drop=False)
zone_months

,region_zone,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves
0,rUP71,201811,366.0,32922.0,1.067803,4.137198e+03
1,rUP71,201812,1600.0,65844.0,3.243872,8.274396e+03
2,rUP71,201902,9241.0,724570.0,11.740762,8.238298e+04
3,rUP71,201903,5515.0,241582.0,3.506571,2.827668e+04
4,rUP71,201910,12648.0,1163056.0,21.173730,1.478166e+05
5,rUP71,201912,4710.0,329520.0,0.949721,4.320007e+04
6,rUP71,202001,5136.0,329520.0,0.881431,4.320007e+04
7,rUP71,202002,20561.0,1426420.0,18.729422,1.973158e+05
8,rUP71,202003,62960.0,1976106.0,7.047458,2.415473e+05
9,rUP71,202004,72533.0,2469200.0,9.361431,3.192217e+05


In [10]:
zone_months['zone_exhaustion'] = zone_months['zone_total_extracted']/zone_months['zone_reserves']*100
zone_months['zone_avg_sA'] = zone_months['ep_sA_reserves']/zone_months['zone_reserves']
zone_months.head(10)

,region_zone,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves,zone_exhaustion,zone_avg_sA
0,rUP71,201811,366.0,32922.0,1.067803,4137.198011,1.111719,0.125667
1,rUP71,201812,1600.0,65844.0,3.243872,8274.396022,2.429986,0.125667
2,rUP71,201902,9241.0,724570.0,11.740762,82382.978088,1.275377,0.113699
3,rUP71,201903,5515.0,241582.0,3.506571,28276.678022,2.282869,0.117048
4,rUP71,201910,12648.0,1163056.0,21.173730,147816.560022,1.087480,0.127093
5,rUP71,201912,4710.0,329520.0,0.949721,43200.072000,1.429352,0.131100
6,rUP71,202001,5136.0,329520.0,0.881431,43200.072000,1.558631,0.131100
7,rUP71,202002,20561.0,1426420.0,18.729422,197315.846000,1.441441,0.138329
8,rUP71,202003,62960.0,1976106.0,7.047458,241547.266154,3.186064,0.122234
9,rUP71,202004,72533.0,2469200.0,9.361431,319221.682110,2.937510,0.129281


In [11]:
zone_months.to_csv('Resources/zone_months.csv',index=False)

In [12]:
region_months = histdf.groupby(['region','year_month']).apply(ep_agg).reset_index(drop=False)
region_months

,region,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves
0,rU,201810,1001.0,329230.0,2.531069,4.806820e+04
1,rU,201811,1720.0,582142.0,5.113422,7.963054e+04
2,rU,201812,9842.0,1053994.0,19.647050,1.547965e+05
3,rU,201901,32907.0,2196210.0,26.560157,3.050351e+05
4,rU,201902,29638.0,2592000.0,25.801039,3.541994e+05
5,rU,201903,11415.0,681042.0,5.870678,9.148116e+04
6,rU,201904,16298.0,2163117.0,19.469721,3.461295e+05
7,rU,201907,10467.0,1800866.0,26.669246,4.566434e+05
8,rU,201908,18216.0,1888858.0,47.597847,5.785316e+05
9,rU,201909,7703.0,1350839.0,15.755612,3.401231e+05


In [13]:
region_months['zone_exhaustion'] = region_months['zone_total_extracted']/region_months['zone_reserves']*100
region_months['zone_avg_sA'] = region_months['ep_sA_reserves']/region_months['zone_reserves']
region_months.head(10)

,region,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves,zone_exhaustion,zone_avg_sA
0,rU,201810,1001.0,329230.0,2.531069,48068.200000,0.304043,0.146002
1,rU,201811,1720.0,582142.0,5.113422,79630.537011,0.295461,0.136789
2,rU,201812,9842.0,1053994.0,19.647050,154796.485022,0.933781,0.146867
3,rU,201901,32907.0,2196210.0,26.560157,305035.101000,1.498354,0.138892
4,rU,201902,29638.0,2592000.0,25.801039,354199.399088,1.143441,0.136651
5,rU,201903,11415.0,681042.0,5.870678,91481.157022,1.676108,0.134325
6,rU,201904,16298.0,2163117.0,19.469721,346129.495033,0.753450,0.160014
7,rU,201907,10467.0,1800866.0,26.669246,456643.351989,0.581220,0.253569
8,rU,201908,18216.0,1888858.0,47.597847,578531.630912,0.964392,0.306286
9,rU,201909,7703.0,1350839.0,15.755612,340123.081923,0.570238,0.251787


In [14]:
region_months.to_csv('Resources/region_months.csv',index=False)